In [ ]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, TimeDistributed, Bidirectional
from keras.layers import LSTM, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers.core import Dropout
from keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import keras
import codecs
import os

%matplotlib inline

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
np.random.seed(42)

In [ ]:
# Загрузка данных в Google Colab
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# file_id = '1-f_PhUmFPfWfBXMdf5fAN5aO2o-wsL-_' #Echo
file_id = '1xTurzpt1s6hMugJNgCnTgfLBo5YXm0Rd' #VK
# file_id = '1QLahcX9SxHL0zyMd48K8cuyNR1pQogyZ' #Habr


import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  _, done = downloader.next_chunk()
downloaded.seek(0)
csv_data = downloaded.read()

In [ ]:
file = io.StringIO(csv_data.decode('utf-8'))

In [ ]:
# Read echo
import sys
import csv

csv.field_size_limit(sys.maxsize)
data = pd.read_csv(file, sep='|', engine='python', index_col=0, error_bad_lines=False);

In [ ]:
# Read VK
data = pd.read_csv(file, names=['Author', 'Text'], sep=',', header=None)

In [ ]:
# Read Habrahabr
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
data.Text = data.Text.apply(str);
data['CountWords'] = data['Text'].map(lambda x: len(x.split()))
print('Количество статей в корпусе:', len(data))

In [ ]:
# Удаление спам пользователей из корпуса VK
spam_list = [301557842, 184330667, 191315693, 111604181, 212846761, 278053852, 281162155, 206300456, 306859957, 19006413, 275267400, 131629784, 276695686, 227968805, 65265920, 215657793, 212066696, 281866527, 172515637, 420657548, 259904924, 103086082, 36309849, 151622711, 304407589, 304407589, 234135027, 330779808, 135431520, 326927985, 22070189, 16497456, 22070189, 16497456, 120772826, 181255568, 24655736, 90966607, 102977537, 317336618, 189704404, 11663845, 57439210, 163452787, 38680483, 121144040, 211289578, 280155968, 198528079, 277512341, 33152119, 314327791, 229101154, 271546215, 4121239, 59729051, 288393060, 17620663, 296431137, 320666372, 153340241, 13309210, 333014765, 24595847, 152263680, 135815313, 27763994, 34902711, 200488318, 393710429, 400853612, 18091069, 232368999, 243123251, 159034443, 101820061, 334397479, 36372315, 18412482, 362494239, 98538332, 94983825, 171941220, 35386477, 179364423, 136668179, 57446172, 288223392, 321431512, 277994902, 190694040, 325021098, 129392296, 289027630, 216216545, 75884932, 208023062, 185965346, 2857932, 2574325, 154034429, 312715518, 286913827, 336049060, 163410643, 309938895, 372204020, 400615680, 235441786, 8391421, 136997064, 174934251, 173179778, 134275738, 93070783, 403039516, 326194938, 26960550, 18090797, 15931375, 64401851, 14665799, 223678239, 1963998, 252135842, 16352364, 386230491, 135103805, 335233013, 226605259, 94018354, 363143703, 278775919, 134780377,498638, 498638, 20403116, 824871, 22259914, 203601767, 386435681, 280519165, 225577927, 336493264, 203836012, 250362235, 135154139, 332671406, 296879545, 191398452, 48411613, 276175277, 77459397, 15155303, 6646818, 174692652, 25755928, 145799445, 210213078, 225680585, 167783297, 159347782, 154637631, 343704548, 276329737, 158954181, 7736592, 52430830, 54738353, 181641220, 249065698, 356566757, 12276180, 146475131, 38034256, 301152537, 181250759, 65169740, 229445819, 360098848, 204181735, 311981137, 9365217, 323032519, 144081373, 166471693, 219977388, 3252582, 92645976, 341675206, 394122042, 8469671, 293599275, 150564713, 315874927, 75299334, 395839270, 256327432, 8758868, 234523373, 234523373] 
print('Count of spam users:', len(spam_list))
lam = lambda row: row['Author'] not in spam_list
data = data[data.apply(lam, axis=1)]

In [ ]:
author_count_news = data.Author.value_counts()
print('Количество авторов в корпусе:', len(author_count_news))

In [ ]:
num_classes = 100
skip_person = 9
author_count_news_dubset = author_count_news[skip_person:skip_person+num_classes]

In [ ]:
author_count_news_dubset.plot(kind='bar', figsize=(15, 5), title="Number of author's articles");

In [ ]:
temp_data = pd.DataFrame()
names_author = author_count_news_dubset.index.values

for author in names_author:
    temp_data = temp_data.append(data[data.Author == author])

data = temp_data
print('Количество статей после удаления:', len(data))

In [ ]:
print('Count Words in text', sum(data['CountWords']))
print('Количество статей в корпусе:', len(data))

In [ ]:
names = data.Author.value_counts().index.values

lableEnc = LabelEncoder()
lableEnc.fit(names.ravel()) 
lables = lableEnc.transform(names).reshape((num_classes, 1))

oneHotEnc = OneHotEncoder()
oneHotEnc.fit(lables)

for author in names:
    val = lableEnc.transform([author])[0]
    data.Author.replace(to_replace=author, value=val, inplace=True)

data.head()

In [ ]:
def get_texts_to_matrix(texts, max_features = 0):
    tokenizer = Tokenizer(split=" ", lower=True)
    if max_features != 0:
        tokenizer = Tokenizer(split=" ", lower=True, num_words=max_features)
    
    tokenizer.fit_on_texts(texts)
    matrix_tfidf = tokenizer.texts_to_matrix(texts=texts, mode='tfidf')
    print('Количество текстов:', matrix_tfidf.shape[0])
    print('Количество токенов:', matrix_tfidf.shape[1])
    return matrix_tfidf

In [ ]:
def get_texts_to_sequences(text):
    tokenizer = Tokenizer(split=" ", lower=True)
    tokenizer.fit_on_texts(text)
    text_sequences = tokenizer.texts_to_sequences(text)
    print('В словаре {} слов'.format(len(tokenizer.word_index)))
    return text_sequences

In [ ]:
def get_texts_to_gramm_sequences(texts, n=3, lowercase=True, remove_p=False):
    """ Convert text into character ngrams. """
    char_filter = '«»—!–\"−#$%&()*…,-./:;<=>?@[\\\\]^_`{|}~\\t\\n'
    vectors = []
    dict_grams = {}
    char_index = 0
    for text in texts:
        if remove_p:
            for c_filter in char_filter:
                text = text.replace(c_filter, '')
        if lowercase:
            text = text.lower()
        vector = []
        grams_for_text = ["".join(j) for j in zip(*[text[i:] for i in range(n)])]
        for gram in grams_for_text:
            if gram in dict_grams:
                vector.append(dict_grams[gram])
            else:
                dict_grams[gram] = char_index
                vector.append(char_index)
                char_index += 1
        vectors.append(vector)
    count_grams = len(dict_grams)
    print('Количество уникальных ngrams:', count_grams)
    return vectors, count_grams

In [ ]:
X, count_grams = get_texts_to_gramm_sequences(data['Text'], n=3)

In [ ]:
def print_plot_mean(x):
    means = [len(z) for z in x]
    plt.plot(means);

In [ ]:
print_plot_mean(X)

In [ ]:
def get_X_y_for_traning(X, y, num_words):
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=num_words)     
    y = keras.utils.to_categorical(y, num_classes)
    print('Размерность X:', X.shape)
    print('Размерность y:', y.shape)
    return X, y

In [ ]:
num_words = 3000

X_full, y_full = get_X_y_for_traning(X, data.Author, num_words)
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

In [ ]:
def get_lstm_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_bidirectional_lstm():
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(20, dropout=0.3, recurrent_dropout=0.3)))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_conv_conv_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_conv_lstm_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.3))
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_lstm_conv_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
    model.add(Conv1D(filters=512, kernel_size=3, activation='sigmoid'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_conv_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_dense_model(embeding_size):
    model = Sequential()
    model.add(Embedding(embeding_size, 200, input_shape=(3000,)))
    model.add(SpatialDropout1D(0.2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dropout (0.2))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout (0.2))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout (0.2))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = get_conv_model(count_grams)
model.summary()

BATCH_SIZE = 32
EPOCHS = 10
VERBOSE = 2

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, 
          verbose=VERBOSE, validation_data=(X_test, y_test))